# End-to-End Twitter Turkish Sentiment Analysis Application
## 2-Sentiment Analysis


### Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/morethan101/')

### Install Packages

In [ ]:
!pip install transformers
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=6c37a37c4165b1d9db96c53c5d1708bf1b638d1ed853b2654cebb431846ed8f1
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji


### Import Packages

In [ ]:
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
import re
import emoji

In [ ]:
df = pd.read_csv("data/deprem_hashtag_subat_2023_unique.csv")
df

,Unnamed: 0,ID,Date,Username,Tweet,ReplyCount,RetweetCount,LikeCount,QuoteCount,URL
0,0,1622746962318577664,2023-02-06 23:59:59+00:00,Brk34587273,İstanbulda gök gürültüsü başladı Allah’ım sen ...,0,1,1,0,https://twitter.com/Brk34587273/status/1622746...
1,1,1622746958363328512,2023-02-06 23:59:58+00:00,SerbestSabuncu,#deprem \nBu gece dualarımız enkaz altında kal...,0,1,1,0,https://twitter.com/SerbestSabuncu/status/1622...
2,2,1622746957792915457,2023-02-06 23:59:58+00:00,zeynepkacar2504,Ülkede afet var fırsatçılar iş başında..!\nÜlk...,0,1,2,0,https://twitter.com/zeynepkacar2504/status/162...
3,3,1622746957172117505,2023-02-06 23:59:58+00:00,Ahmetusmann,ARKADAŞIMIZDAN HABER ALAMIYORUZ ACİLEN RT ATIP...,11,1102,571,9,https://twitter.com/Ahmetusmann/status/1622746...
4,4,1622746954118619137,2023-02-06 23:59:57+00:00,aligsderya,Askerlerimiz kışladan çıksın Yollar KAPALI z...,0,0,7,0,https://twitter.com/aligsderya/status/16227469...
...,...,...,...,...,...,...,...,...,...,...
19802,19995,1622721663790985217,2023-02-06 22:19:28+00:00,AruobaB,İnanamıyorum insanlarımız ölüyo hem de saçma s...,0,0,0,0,https://twitter.com/AruobaB/status/16227216637...
19803,19996,1622721663631589376,2023-02-06 22:19:27+00:00,halit_tazeoglu,Dışarıda soğukta kalan varsa Gaziantep Ortadoğ...,1,0,1,0,https://twitter.com/halit_tazeoglu/status/1622...
19804,19997,1622721662624956416,2023-02-06 22:19:27+00:00,anildegilim,@grahamlicc ‼️ arkadasimin halasindan 20 saatt...,0,0,0,0,https://twitter.com/anildegilim/status/1622721...
19805,19998,1622721662306189317,2023-02-06 22:19:27+00:00,BayogluIsrafil,"Biz ne ara bu kadar kötü, çıkarcı, vefasız ve ...",1,0,6,0,https://twitter.com/BayogluIsrafil/status/1622...


### Sentiment Analysis Model init

In [ ]:
#load model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("savasy/bert-base-turkish-sentiment-cased")
tokenizer = AutoTokenizer.from_pretrained("savasy/bert-base-turkish-sentiment-cased")
# create pipeline
sa = pipeline("sentiment-analysis", tokenizer=tokenizer, model=model)

### Preprocessing

In [ ]:
def preprocess_word(word):
    # Remove punctuation
    #word = word.strip('\'"?!,.():;')
    
    # Convert more than 2 letter repetitions to 2 letter
    # funnnnny --> funny
    word = re.sub(r'(.)\1+', r'\1\1', word)
    # Remove - & '
    word = re.sub(r'(-|\')', '', word)
    return word


def is_valid_word(word):
    # Check if word begins with an alphabet
    return (re.search(r'^[a-zA-Z][a-z0-9A-Z\._]*$', word) is not None)


def handle_emojis(tweet):
    # Smile -- :), : ), :-), (:, ( :, (-:, :')
    tweet = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', ' EMO_POS ', tweet)
    # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
    tweet = re.sub(r'(:\s?D|:-D|x-?D|X-?D)', ' EMO_POS ', tweet)
    # Love -- <3, :*
    tweet = re.sub(r'(<3|:\*)', ' EMO_POS ', tweet)
    # Wink -- ;-), ;), ;-D, ;D, (;,  (-;
    tweet = re.sub(r'(;-?\)|;-?D|\(-?;)|😉', ' EMO_POS ', tweet)
    # Sad -- :-(, : (, :(, ):, )-:
    tweet = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', ' EMO_NEG ', tweet)
    # Cry -- :,(, :'(, :"(
    tweet = re.sub(r'(:,\(|:\'\(|:"\()', ' EMO_NEG ', tweet)

    return tweet

def remove_emoji(tweet):
    return emoji.replace_emoji(tweet, replace=" ")


def preprocess_tweet(tweet):
    processed_tweet = []
    # Convert to lower case
    tweet = tweet.lower()
    
    #Clean only digits
    tweet = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", tweet)
    
    # Replaces URLs with the word URL
    #tweet = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', ' URL ', tweet)
    tweet = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', '', tweet)
    
    # Replace @handle with the word USER_MENTION
    #tweet = re.sub(r'@[\S]+', 'USER_MENTION', tweet)
    tweet = re.sub(r'@[\S]+', '', tweet)
    
    # Replaces #hashtag with hashtag
    #tweet = re.sub(r'#(\S+)', r' \1 ', tweet)
    tweet = re.sub(r'#(\S+)', '', tweet)
    
    # Remove RT (retweet)
    tweet = re.sub(r'\brt\b', '', tweet)
    
    # Replace 2+ dots with space
    tweet = re.sub(r'\.{2,}', ' ', tweet)
    
    # Strip space, " and ' from tweet
    tweet = tweet.strip(' "\'')

    # Replace emojis with either EMO_POS or EMO_NEG
    #tweet = handle_emojis(tweet)

    #remove emojis
    tweet = remove_emoji(tweet)
    
    # Replace multiple spaces with a single space
    tweet = re.sub(r'\s+', ' ', tweet)
    words = tweet.split()

    for word in words:
      word = preprocess_word(word)
      #if is_valid_word(word):
      #    processed_tweet.append(word)
      processed_tweet.append(word)

    return ' '.join(processed_tweet)

In [ ]:
index = 2000
print(df.loc[index].Tweet)
p = sa(df.loc[index].Tweet)
print(p)

#deprem korkuyorum donarak ölecekler!!! Korkuyorum kurtarılmayı beklerken kurtarılmadan ölecekler!!!! Yarab yardım et. @OguzhanUgur
[{'label': 'negative', 'score': 0.8632420301437378}]


In [ ]:
pre_twit = preprocess_tweet(df.loc[index].Tweet)
print(pre_twit)
p = sa(pre_twit)
print(p)

korkuyorum donarak ölecekler!! korkuyorum kurtarılmayı beklerken kurtarılmadan ölecekler!! yarab yardım et.
[{'label': 'negative', 'score': 0.9035896062850952}]


In [ ]:
def get_sentiment_analysis(tweet):
    """True if tweet has positive compound sentiment, False otherwise."""
    p = sa(str(tweet))
    #print("label", p[0]["label"])
    #print("score", round(p[0]["score"],4))
    return pd.Series([p[0]["label"], round(p[0]["score"],4)])

In [ ]:
s = get_sentiment_analysis(pre_twit)
print(s)

0    negative
1      0.9036
dtype: object


### Sentiment Analysis

In [ ]:
df["sentiment_label"] = "-"
df["sentiment_score"] = -1
df[["sentiment_label", "sentiment_score"]] = df["Tweet"].progress_apply(get_sentiment_analysis)

  0%|          | 0/19807 [00:00<?, ?it/s]

In [ ]:
df

,Unnamed: 0,ID,Date,Username,Tweet,ReplyCount,RetweetCount,LikeCount,QuoteCount,URL,sentiment_label,sentiment_score
0,0,1622746962318577664,2023-02-06 23:59:59+00:00,Brk34587273,İstanbulda gök gürültüsü başladı Allah’ım sen ...,0,1,1,0,https://twitter.com/Brk34587273/status/1622746...,negative,0.9343
1,1,1622746958363328512,2023-02-06 23:59:58+00:00,SerbestSabuncu,#deprem \nBu gece dualarımız enkaz altında kal...,0,1,1,0,https://twitter.com/SerbestSabuncu/status/1622...,positive,0.8156
2,2,1622746957792915457,2023-02-06 23:59:58+00:00,zeynepkacar2504,Ülkede afet var fırsatçılar iş başında..!\nÜlk...,0,1,2,0,https://twitter.com/zeynepkacar2504/status/162...,negative,0.9911
3,3,1622746957172117505,2023-02-06 23:59:58+00:00,Ahmetusmann,ARKADAŞIMIZDAN HABER ALAMIYORUZ ACİLEN RT ATIP...,11,1102,571,9,https://twitter.com/Ahmetusmann/status/1622746...,negative,0.9968
4,4,1622746954118619137,2023-02-06 23:59:57+00:00,aligsderya,Askerlerimiz kışladan çıksın Yollar KAPALI z...,0,0,7,0,https://twitter.com/aligsderya/status/16227469...,positive,0.8603
...,...,...,...,...,...,...,...,...,...,...,...,...
19802,19995,1622721663790985217,2023-02-06 22:19:28+00:00,AruobaB,İnanamıyorum insanlarımız ölüyo hem de saçma s...,0,0,0,0,https://twitter.com/AruobaB/status/16227216637...,negative,0.9907
19803,19996,1622721663631589376,2023-02-06 22:19:27+00:00,halit_tazeoglu,Dışarıda soğukta kalan varsa Gaziantep Ortadoğ...,1,0,1,0,https://twitter.com/halit_tazeoglu/status/1622...,negative,0.5360
19804,19997,1622721662624956416,2023-02-06 22:19:27+00:00,anildegilim,@grahamlicc ‼️ arkadasimin halasindan 20 saatt...,0,0,0,0,https://twitter.com/anildegilim/status/1622721...,negative,0.8692
19805,19998,1622721662306189317,2023-02-06 22:19:27+00:00,BayogluIsrafil,"Biz ne ara bu kadar kötü, çıkarcı, vefasız ve ...",1,0,6,0,https://twitter.com/BayogluIsrafil/status/1622...,negative,0.9989


In [ ]:
df_positive = df[df["sentiment_label"]=="positive"]
df_positive

,Unnamed: 0,ID,Date,Username,Tweet,ReplyCount,RetweetCount,LikeCount,QuoteCount,URL,sentiment_label,sentiment_score
1,1,1622746958363328512,2023-02-06 23:59:58+00:00,SerbestSabuncu,#deprem \nBu gece dualarımız enkaz altında kal...,0,1,1,0,https://twitter.com/SerbestSabuncu/status/1622...,positive,0.8156
4,4,1622746954118619137,2023-02-06 23:59:57+00:00,aligsderya,Askerlerimiz kışladan çıksın Yollar KAPALI z...,0,0,7,0,https://twitter.com/aligsderya/status/16227469...,positive,0.8603
5,5,1622746952893976578,2023-02-06 23:59:57+00:00,hamzacak,"Gün suçlu arama; sağa veya sola çamur atma, ik...",0,0,0,0,https://twitter.com/hamzacak/status/1622746952...,positive,0.9234
6,6,1622746949722963969,2023-02-06 23:59:56+00:00,Haruun_,"Keşke, Keşkee, Keşkeee 🙏🙄\n#deprem #TSKsahaya ...",0,0,0,0,https://twitter.com/Haruun_/status/16227469497...,positive,0.9021
7,7,1622746948225716225,2023-02-06 23:59:56+00:00,asivesahsi,SİTE 1 NO:20 HATAY MERKEZ MELİSA YARDIM BEKLİY...,0,0,0,0,https://twitter.com/asivesahsi/status/16227469...,positive,0.6985
...,...,...,...,...,...,...,...,...,...,...,...,...
19791,19984,1622721673764929537,2023-02-06 22:19:30+00:00,FatmaKahriman18,"Haber alamıyoruz lütfen yayalım, ulaştıralım...",0,6,2,0,https://twitter.com/FatmaKahriman18/status/162...,positive,0.8268
19793,19986,1622721672699674626,2023-02-06 22:19:30+00:00,LiderHaberAjans,Kars’tan deprem bölgesine 16 iş makinesi gönde...,5,27,352,2,https://twitter.com/LiderHaberAjans/status/162...,positive,0.8970
19794,19987,1622721670594142208,2023-02-06 22:19:29+00:00,iy26022010,Şakamı abi #deprem #hataydepremi #antakya #AFA...,0,0,0,0,https://twitter.com/iy26022010/status/16227216...,positive,0.5310
19800,19993,1622721666102005760,2023-02-06 22:19:28+00:00,kaaa_kkkk,Malatya'da çalışmalar devam ediyor #deprem htt...,0,0,0,0,https://twitter.com/kaaa_kkkk/status/162272166...,positive,0.7261


In [ ]:
df_positive.to_csv("data/deprem_hashtag_subat_2023_positive.csv")

In [ ]:
df_negative = df[df["sentiment_label"]=="negative"]
df_negative

,Unnamed: 0,ID,Date,Username,Tweet,ReplyCount,RetweetCount,LikeCount,QuoteCount,URL,sentiment_label,sentiment_score
0,0,1622746962318577664,2023-02-06 23:59:59+00:00,Brk34587273,İstanbulda gök gürültüsü başladı Allah’ım sen ...,0,1,1,0,https://twitter.com/Brk34587273/status/1622746...,negative,0.9343
2,2,1622746957792915457,2023-02-06 23:59:58+00:00,zeynepkacar2504,Ülkede afet var fırsatçılar iş başında..!\nÜlk...,0,1,2,0,https://twitter.com/zeynepkacar2504/status/162...,negative,0.9911
3,3,1622746957172117505,2023-02-06 23:59:58+00:00,Ahmetusmann,ARKADAŞIMIZDAN HABER ALAMIYORUZ ACİLEN RT ATIP...,11,1102,571,9,https://twitter.com/Ahmetusmann/status/1622746...,negative,0.9968
9,9,1622746946325577728,2023-02-06 23:59:55+00:00,KbraKoy98945001,Adıyaman yardım bekliyor nolur enkaz altınlard...,0,0,2,0,https://twitter.com/KbraKoy98945001/status/162...,negative,0.9456
10,10,1622746945784512514,2023-02-06 23:59:55+00:00,Denzi28,#hatayyardimbekliyor #deprem #foxtv @ahbap @De...,1,2,4,0,https://twitter.com/Denzi28/status/16227469457...,negative,0.9982
...,...,...,...,...,...,...,...,...,...,...,...,...
19802,19995,1622721663790985217,2023-02-06 22:19:28+00:00,AruobaB,İnanamıyorum insanlarımız ölüyo hem de saçma s...,0,0,0,0,https://twitter.com/AruobaB/status/16227216637...,negative,0.9907
19803,19996,1622721663631589376,2023-02-06 22:19:27+00:00,halit_tazeoglu,Dışarıda soğukta kalan varsa Gaziantep Ortadoğ...,1,0,1,0,https://twitter.com/halit_tazeoglu/status/1622...,negative,0.5360
19804,19997,1622721662624956416,2023-02-06 22:19:27+00:00,anildegilim,@grahamlicc ‼️ arkadasimin halasindan 20 saatt...,0,0,0,0,https://twitter.com/anildegilim/status/1622721...,negative,0.8692
19805,19998,1622721662306189317,2023-02-06 22:19:27+00:00,BayogluIsrafil,"Biz ne ara bu kadar kötü, çıkarcı, vefasız ve ...",1,0,6,0,https://twitter.com/BayogluIsrafil/status/1622...,negative,0.9989


In [ ]:
df_negative.to_csv("data/deprem_hashtag_subat_2023_negative.csv")